# Simulación de flujo en Colmena

# Logística 

Variables a considerar: 

1. Costos de flete por centímetro cúbico en cada uno de los vehículos con los que se cuentan. 
2. Capacidad de los vehículos. 
3. Spread que tenemos para cada punto y temporalidad de los pedidos. 

Con las variables 1,2,3 se determina cuando se tiene que mandar la notificación al usuario y con qué descuento dependiendo de las urgencias de los pedidos. 
Se juega con el spread de los productos para poder llenar el vehículo de transporte. 

Cada producto considera: el volumen y su peso para saber cuál es la fracción que representa del vehículo en cuestión. 

Consideraciones:  

1. Medición de productos y pesos específicos. 
2. User personas por producto (clusters). 
3. Vehículos: espacios de camiones específico, límite mínimo y máximo, especificación de velocidad del vehículo, detalles técnicos del vehículo (como la antiguedad). 

Bodega: 

1. Cedis MVP: Detalles de espacios y capacidades y tiempos de apertura de bodegas/cedis. 
2. Cada casa se busca que sea una bodega. 

Medición a futuro: 

1. Límites de espacios y pesos de casas para almacenamiento. 
2. Espacios y pesos límite por bodega que no sea casa. 

Logística: 
1. Optimización de rutas. 
2. Optimización de puntos de entrega contra availability dentro de espacios en el camión. 
3. Sensores para medir eficiencias en tiempo real. 

Logística a última milla: 
1. Se va a cobrar extra (o simular menos descuento) so se entrega a tu casa. 

En las rutas: 
1. Ruta de punto A-B más eficiente considerando en donde se encuentra el camión, el gas que le queda, y las llantas. 

Tiempos de entrega: 
1. Prevender productos abajo de precio de mercado (costo para nosotros de producto + flete de punto A a punto B y flete de última milla + CAC = o menor precio de venta de supermercado). 
2. Estimar fecha de entrega donde se estima prevender todo el camión lleno (de esto sí depende del tipo de vehículo que se usa). 

Proveedores: 
1. Checar con rutas específicas qué proveedores abastecen la demanda de X productos para Y puntos de entrega. 

Costos:  
1. Linkear los costos de producto + flete + ...
2. Partir de un costo mínimo (costo producto (dependiendo cantidad comprada) + rutas )

Venta: 
1. Precios de venta abajo de mercado. 
2. Precio de venta con utilidad.
3. Precio de venta final-Costo (con seguro de imprevistos de venta). 

Reward:

1. Minigames, 
2. Cupones, 
3. Descuentos directos, 
4. Clusterización de users ideales por producto para más descuento, 
5. Referral programs para descuento. 


Usuarios: 
Van a recibir una notificación para unirse a una compra cuando el algoritmo indique que son posibles compradores y calculando el posible precio al que se le va a ofrecer al cliente. 

# Diseño del sistema de simulación 

Consideraciones 

1. Se simulará una base de datos con M productos que tengan las variables de dimensión, precio y peso. 
2. Se simulará una base de datos con camiones ficticios que presenten las variables de capacidad mínima y máxima, porcentaje de tanque lleno y ubicación actual. 
3. Se simulará una ubicación aleatoria que corresponde al punto de distribución de la bodega. 
4. Se simularán usuarios que puedan o no decidir si compran un producto y que tengan asignada una categoría resultante de la clusterización. 

# Algoritmos auxiliares 

1. Algoritmo que simule el envío de notificaciones de usuarios dependiendo el cluster al que estás asignados. 
2. Cálculo de las ganancias con base en la oferta, demanda, y consideraciones generales de cantidad de producto por camión, cantidad de usuarios que lo adquirieron. 
3. Algoritmo que grafique de manera dinámica los ingresos por ventas y por camión, dependiendo los productos que se han colocado. 
4. Algoritmo que simule el reward system, es deir, que simule cuántos de los usuarios que adquirieron productos del camión jugaron un videojuego, cuántos registraron cupones o descuentos directos. Y que pase esta información para también graficar los ingresos por ventas después de haber aplicado los respectivos descuentos. 
5. Algoritmo de planeación de ruta de punto A-B (estimación de tiempos) para tener mapeado el tiempo de espera de llegada del camión. 

In [2]:
import numpy as np
import random 
# Algoritmo que simula la cantidad de clientes que solicitan un producto determinado
# en un periodo de tiempo determinado (utilizamos una variable aleatoria de Poisson)
'''
@param: lambda = tasa de llegada de clientes en un periodo de tiempo determinado
'''
def simularDemanda(lambda_):
    return np.random.poisson(lambda_)

# Función que simula si un cliente compra un producto o no (utilizamos una variable aleatoria de Bernoulli)
'''
@param: p = probabilidad de que un cliente compre un producto
'''
def simularCompra(p):
    return np.random.binomial(1, p)

# Función que simula la tasa con la que se darán descuentos a los clientes (utilizamos una beta con parámetros 0.5, 0.5)
def simularTasaDescuento(a, b):
    # Se puede ver como un descuento personalizado a medida que se simula de una distribución beta para cada cliente
    # Si los descuentos son estáticos, se modifica esta función para que de acuerdo a un diccionario regrese un valor predeterminado 
    return np.random.beta(a, b)

In [3]:
# Construcción del objeto producto
class Producto:
    def __init__(self, id_, nombre, precio, peso, dimensiones):
        self.id_ = id_
        self.nombre = nombre
        self.precio = precio
        self.peso = peso
        self.dimensiones = dimensiones
    
    def __str__(self):
        return 'Producto: ' + str(self.id_) + ' Nombre: ' + str(self.nombre) + ' Precio: ' + str(self.precio) + ' Peso: ' + str(self.peso) + ' Dimensiones: ' + str(self.dimensiones)


# Construcción del objeto camión 
class Camion:
    def __init__(self, id_, capacidad, modelo):
        self.id_ = id_
        self.modelo = modelo 
        self.capacidad = capacidad
        self.carga = 0
        self.productos = []
    
    # Método que agrega un producto al camión
    def agregarProducto(self, producto):
        if self.carga + producto.peso <= self.capacidad:
            self.productos.append(producto)
            self.carga += producto.peso
            return True
        else:
            return False
    
    def __str__(self):
        return 'Camion: ' + str(self.id_) + ' Carga: ' + str(self.carga) + ' Productos: ' + str(self.productos)
    
# Construcción del objeto cliente
class Cliente:
    def __init__(self, id_, nombre, direccion, telefono, cluster):
        self.id_ = id_
        self.nombre = nombre
        self.direccion = direccion
        self.telefono = telefono
        # clave (id del producto): value (cantidad de productos) 
        self.productos = {}
        self.cluster = cluster 
        # key (id del producto en descuento): value (valor del descuento)
        self.metodos_descuento = {}
    
    # Método que agrega un producto al cliente
    def agregarProducto(self, producto, cantidad):
        if(producto in self.productos):
            self.productos[producto] += cantidad
        else: 
            self.productos[producto] = cantidad
    
    # Método que agrega un método de descuento al cliente
    def agregarMetodoDescuento(self, id, valor):
        if(id in self.productos): 
            self.metodos_descuento[id] = valor
    
    def __str__(self):
        return 'Cliente: ' + str(self.id_) + ' Nombre: ' + str(self.nombre) + ' Productos: ' + str(self.productos) + ' Métodos de descuento: ' + str(self.metodos_descuento)

In [4]:
# Algoritmos que simulan listas de clientes que compran productos
def simulaClientes(lambda_): 
    clientes = []
    for i in range(simularDemanda(lambda_)):
        clientes.append(Cliente(i, 'Cliente ' + str(i), 'Direccion ' + str(i), 'Telefono ' + str(i), np.random.randint(1, 5)))
    return clientes

# Función para aplicar un descuento a un cliente 
def aplicaDescuentoIndividual(cliente, a, b, producto_descuento): 
    tipos_descuento = ["VideoJuego", "Cupón", "Trivia", "Sorteo"]
    metodo = random.choice(tipos_descuento)
    # Vamos a simular si el cliente posee un método de descuento o no 
    tasa_descuento = simularTasaDescuento(a, b)
    # Caso en el que se puede aplicar el descuento
    if(random.random() < tasa_descuento):
        # Obtenemos el valor del descuento de forma aleatoria
        valor_descuento = simularTasaDescuento(a, b)
        # Agregamos el método de descuento al cliente
        cliente.agregarMetodoDescuento(producto_descuento, valor_descuento)
        return valor_descuento, metodo 
    return 0, None

In [5]:
# Función para calcular los ingresos por ventas considerando que se han aplicado descuentos por venta
# @param: dict (DESCUENTOS) para cada producto 
def calculaIngresosConDescuento(DESCUENTOS, productos):
    ingresos = 0 
    ingresos_por_producto = {}
    for producto, precio in zip(productos['Nombre'], productos['precio']):
        if(producto in DESCUENTOS):
            # Obtenemos los items que no tienen descuento 
            # Vamos a crear un vector de 1's de tamaño len(DESCUENTOS[producto])
            #                   precio_i * sum( (1- descuento_i)) * numero_descuentos_aplicados + items_sin_descuento * precio_i
            ingresos_producto = precio * sum(DESCUENTOS[producto])
            ingresos = ingresos + ingresos_producto
            ingresos_por_producto[producto] = ingresos_producto
        else: 
            # Raise error de que diseño 
            raise Exception("Error de diseño")

    return ingresos, ingresos_por_producto

In [6]:
# Función para calcular el espacio del cargamento (se basa con la columna de productos_solicitados, peso y dimensiones)
def calculaEspacioCargamento(productos):
    espacio = 0
    peso_total = 0 
    for cantidad, peso, dimensiones in zip(productos['productos_solicitados'], productos['peso'], productos['dimensiones']):
        # Sumamos la cantidad de los productos por el volumen (dimensiones)
        espacio +=cantidad * dimensiones
        # Sumamos la cantidad de los productos por el peso
        peso_total += cantidad * peso
    return espacio, peso_total

In [7]:
# Función para ajustar el precio de un producto a medida que su demanda aumenta 
def ajusta_precio(precio, cantidad_nueva): 
    return precio * (1 - cantidad_nueva / 1000) 

## Simulación (MAIN NODE)

In [8]:
# Vamos a construir una base de datos aleatoria con productos que tengan la variable de 
# precio, peso y dimensiones
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display


# demanda_clientes establece el número de clientes que van a demandar un producto, más no la cantidad (esa se define posteriormente)
productos = pd.DataFrame(columns=['Id', 'Nombre', 'precio', 'peso', 'dimensiones', 'cantidad', 'demanda_clientes', 'productos_solicitados'])
nombres = ['Frijoles', 'Leche', 'Cereal', 'Arroz', 'Fruta', 'Jugos']

for i in range(len(nombres)):
    productos.loc[i] = [i, nombres[i], np.random.randint(10, 30), np.random.randint(5, 20), np.random.randint(1, 10), np.random.randint(750, 1000), 0, 0]

# Para determinar la cantidad de productos vendidos vamos a simular la demanda de cada producto por separado 
display(productos)

,Id,Nombre,precio,peso,dimensiones,cantidad,demanda_clientes,productos_solicitados
0,0,Frijoles,10,12,6,923,0,0
1,1,Leche,25,12,9,824,0,0
2,2,Cereal,20,17,8,949,0,0
3,3,Arroz,17,18,1,858,0,0
4,4,Fruta,21,13,6,841,0,0
5,5,Jugos,24,16,9,784,0,0



# Observaciones: 

Con base en el objetivo de ventas, se tiene que llegar al mínimo de producto solicitado para extender descuentos. 


# Variables de control 

In [9]:
# Parámetros globales 
tasa_clientes_compran = 150 # En promedio, 100 clientes están solicitando productos para un camión dado 
# Configuración de la tasa de descuentos 
# La tasa de descuentos está dada por forma_a / (forma_a + forma_b), de esta manera, 3 / (3+6) = 0.3 => 30% de los clientes que compran tienen un descuento
forma_a = 3 
forma_b = 6
# Arreglo que estima la cantidad promedio que se compra por producto 
cantidad_promedio = {}
for producto in productos["Nombre"]: 
    #                                Cantidad promedio que se compra por producto (estimación)
    cantidad_promedio[producto] = np.random.randint(2, 5)

# Ahora configuramos la tasa de demanda para cada producto 
tasas = [np.random.randint(1, tasa_clientes_compran/2) for i in range(len(productos["Nombre"]))]

# Configuramos las tasas de *NUEVOS* clientes que compran por producto
tasa_clientes_compran_nuevos = 2
tasas_nuevos_clientes = {}
for producto in productos["Nombre"]:
    tasas_nuevos_clientes[producto] = np.random.randint(1, tasa_clientes_compran_nuevos)

for producto in range(len(productos["Nombre"])):
    productos.loc[producto, 'demanda_clientes'] = simularDemanda(tasas[producto])

tiempo = 85 # Suponemos que el tiempo de llegada del camión es de 85 minutos

# Esta variable representa el número de clientes que se pueden alcanzar (en una proximidad de los clientes que ya compraron) y convencerlos de comprar a través de publicidad y adds dirigidos
tasa_tota_nuevos_clientes = 30

# Establecemos un limite inferior del precio para cada producto (que es el precio del producto menos el 10%)
limite_inferior = {}
for producto, precio in zip(productos['Nombre'], productos['precio']):
    limite_inferior[producto] = precio - (precio * 0.1)
productos['limite_inferior'] = limite_inferior.values()

# Vamos a construir un dataframe que contenga Nombre, cantidad_promedio, tasas_nuevos_cliente, demanda_clientes (de productos) 
datos = pd.DataFrame(columns=['Nombre', 'cantidad_promedio', 'tasas_nuevos_clientes', 'demanda_clientes'])
datos['Nombre'] = productos['Nombre']
datos['cantidad_promedio'] = cantidad_promedio.values()
datos['tasas_nuevos_clientes'] = tasas_nuevos_clientes.values()
datos['demanda_clientes'] = productos['demanda_clientes']
datos['precio'] = productos['precio']
datos['limite_inferior'] = productos['limite_inferior']
print(cantidad_promedio)
print(tasas_nuevos_clientes)

print(f'La tasa de clientes que compran es de {tasa_clientes_compran}')
print(f'La tasa de clientes que compran (por intervalo de tiempo) nuevos es de {tasa_clientes_compran_nuevos}')
print(f'La tasa de clientes que compran (en general) nuevos es de {tasa_tota_nuevos_clientes}')

datos


{'Frijoles': 2, 'Leche': 2, 'Cereal': 3, 'Arroz': 3, 'Fruta': 4, 'Jugos': 4}
{'Frijoles': 1, 'Leche': 1, 'Cereal': 1, 'Arroz': 1, 'Fruta': 1, 'Jugos': 1}
La tasa de clientes que compran es de 150
La tasa de clientes que compran (por intervalo de tiempo) nuevos es de 2
La tasa de clientes que compran (en general) nuevos es de 30


,Nombre,cantidad_promedio,tasas_nuevos_clientes,demanda_clientes,precio,limite_inferior
0,Frijoles,2,1,22,10,9.0
1,Leche,2,1,77,25,22.5
2,Cereal,3,1,6,20,18.0
3,Arroz,3,1,44,17,15.3
4,Fruta,4,1,11,21,18.9
5,Jugos,4,1,52,24,21.6


# Main simulation

In [16]:
# Estructura que contiene los precios ajustados con los descuentos
DESCUENTOS_APLICADOS = {}
# Estructura que contiene la cantidad solicitada por producto 
CANTIDADES_SOLICITADAS = {}
# Estructura que contiene la cantidad por producto que fue demandado y que no se pudo satisfacer (por agotamiento de inventario)
CANTIDADES_NO_SATISFECHAS = {}
# Estructura que cuenta los métodos de descuento utilizados 
METODOS_DESCUENTO = {}
# El precio del producto va a ir disminuyendo a medida que hay más clientes que lo soliciten 

# 1. Se simulan los clientes que están en un periodo determinado usando la app para comprar (CLIENTES)
CLIENTES = simulaClientes(tasa_clientes_compran)
# 2. Para cada producto,de los clientes simulados, se simula la cantidad de clientes que lo van a comprar y se muestrea esa cantidad de clientes de (CLIENTES)
for producto, demanda in zip(productos["Nombre"], productos["demanda_clientes"]): 
    # Agregamos el producto a los descuentos aplicados
    DESCUENTOS_APLICADOS[producto] = []
    # Agregamos el producto a las cantidades solicitadas
    CANTIDADES_SOLICITADAS[producto] = 0 
    # Agregamos el producto a las cantidades no satisfechas
    CANTIDADES_NO_SATISFECHAS[producto] = 0
    # Vamos a muestrear sin reemplazo de CLIENTES 
    clientes_compran = random.sample(CLIENTES, demanda)
    # Extraemos la cantidad con la que se cuenta de ese producto 
    cantidad_del_producto = productos.loc[productos['Nombre'] == producto, 'cantidad'].values[0]
    # 2.1. Para cada cliente se simula el método de descuento que se le va a aplicar
    for cliente in clientes_compran:
        # Agregamos el producto al cliente 
        # Simulamos de una uniforme discreta de 0 a cantidad_promedio[producto]
        cantidad = np.random.randint(1, cantidad_promedio[producto])
        if(CANTIDADES_SOLICITADAS[producto] + cantidad <= cantidad_del_producto): 
            # Se agrega el producto a la canasta del cliente 
            cliente.agregarProducto(producto, cantidad)
            # Aplicamos el descuento 
            descuento, tipo_descuento = aplicaDescuentoIndividual(cliente, forma_a, forma_b, producto)
            # Vamos a guardar el descuento en un arreglo de descuentos para calcular posteriormente los ingresos por ventas (ya con el descuento aplicado)
            # Este arreglo nos facilitará el cálculo de los descuentos por producto para no iterar por todos los clientes
            DESCUENTOS_APLICADOS[producto].append(cantidad * (1 - descuento))
            # Agregamos las cantidades solicitadas 
            CANTIDADES_SOLICITADAS[producto] += cantidad
            # Vamos a registrar el método por descuento 
            if(tipo_descuento in METODOS_DESCUENTO):
                METODOS_DESCUENTO[tipo_descuento] += 1
            elif(tipo_descuento != None):
                METODOS_DESCUENTO[tipo_descuento] = 1
        else:
            print(f'Se han agotado las existencias del producto {producto}')
            CANTIDADES_NO_SATISFECHAS[producto] += cantidad

    # Ajustamos los productos solicitados al dataframe productos 
    productos.loc[productos["Nombre"] == producto, "productos_solicitados"] = CANTIDADES_SOLICITADAS[producto]

# Imprimimos los métodos de descuento que los clientes aplicaron durante la simulación 
print(f'Métodos de descuento aplicados: {METODOS_DESCUENTO}')
# Vamos a hacer una piechart utilizando plotly para desplegar los métodos de descuento
fig = go.Figure(data=[go.Pie(labels=list(METODOS_DESCUENTO.keys()), values=list(METODOS_DESCUENTO.values()))], layout=go.Layout(title="Métodos de descuento aplicados"))
# Cabiamos la tiografía de la gráfica
fig.update_layout(font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"))
fig.show()

# 2.2 Se calculan los ingresos de ventas en total y por producto ya considerando los descuentos y cantidad de personas que solicitan cada producto 
ingresos, ingresos_x_producto = calculaIngresosConDescuento(DESCUENTOS_APLICADOS, productos)
print(f'Ingresos totales: {ingresos} y por producto: {ingresos_x_producto}')

# Vamos a hacer un piechart utilizando plotly para desplegar los ingresos por producto
fig = go.Figure(data=[go.Pie(labels=list(ingresos_x_producto.keys()), values=list(ingresos_x_producto.values()))], layout=go.Layout(title="Ingresos por producto"))
fig.update_layout(font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"))
fig.show()

# 3. Dependiendo del total de productos demandados, se va a escoger un camión que cuente con la capacidad suficiente para transportarlos (CAMIONES)
espacio, peso_total = calculaEspacioCargamento(productos)
print(f'Espacio total: {espacio} y peso total: {peso_total}')

# 4. Gráfica de los productos no satisfechos (de barra y por producto) de CANTIDADES_NO_SATISFECHAS 
fig = go.Figure(data=[go.Bar(x=list(CANTIDADES_NO_SATISFECHAS.keys()), y=list(CANTIDADES_NO_SATISFECHAS.values()))], layout=go.Layout(title="Cantidad de productos no satisfechos"))
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6)
fig.update_layout(font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"))
fig.show()

# Vamos a hacer una gráfica de barras para desplegar la cantidad de productos solicitados 
fig = go.Figure(data=[go.Bar(x=list(CANTIDADES_SOLICITADAS.keys()), y=list(CANTIDADES_SOLICITADAS.values()))], layout=go.Layout(title="Cantidad de productos solicitados"))
# Agregamos diferentes colores a la gráfica
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6)
fig.update_layout(font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"))
fig.show()

display(productos)

Métodos de descuento aplicados: {'Trivia': 16, 'Sorteo': 19, 'Cupón': 15, 'VideoJuego': 14}


Ingresos totales: 5292.06021061655 y por producto: {'Frijoles': 182.94077837052774, 'Leche': 1647.0755136361029, 'Cereal': 142.08586210992792, 'Arroz': 909.3076830784929, 'Fruta': 318.8626760316768, 'Jugos': 2091.7876973898206}


Espacio total: 2060 y peso total: 4461


,Id,Nombre,precio,peso,dimensiones,cantidad,demanda_clientes,productos_solicitados,limite_inferior
0,0,Frijoles,9.047386,12,6,923,22,22,9.0
1,1,Leche,23.099367,12,9,824,77,77,22.5
2,2,Cereal,18.424111,17,8,949,6,9,18.0
3,3,Arroz,15.738985,18,1,858,44,62,15.3
4,4,Fruta,18.980721,13,6,841,11,20,18.9
5,5,Jugos,22.020210,16,9,784,52,109,21.6


In [11]:
# Estructuras de datos que contendrán el cambio en el tiempo de las variables de interés 
cantidades_solicitadas_en_tiempo = {}
for producto in productos["Nombre"]:
    # La cantidad_t0 es la cantidad de productos solicitados del dataframe de productos (antes de mandar el camión )
    cantidad_t0 = productos[productos["Nombre"] == producto]["productos_solicitados"].tolist()[0]
    cantidades_solicitadas_en_tiempo[producto] = [cantidad_t0]

# Estructura que contiene los precios por producto en el tiempo
precios_por_producto_en_tiempo = {}
for producto in productos["Nombre"]:
    precio = productos[productos["Nombre"] == producto]["precio"].tolist()[0]
    precios_por_producto_en_tiempo[producto] = [precio]

# Estructura que contiene los ingresos por producto en el tiempo 
ingresos_por_producto_en_tiempo = {}
for producto in productos["Nombre"]:
    ingreso = ingresos_x_producto[producto]
    ingresos_por_producto_en_tiempo[producto] = [ingreso]

clientes_nuevos_tiempo = []
ingresos_en_tiempo = [ingresos]
cantidad_de_descuentos_aplicados_tiempo = []

In [12]:

# 4. Se establece el punto inicial de la ruta y el punto final de la ruta (RUTAS) (Conexión con Google Maps para estimar el tiempo de llegada) (DONE)

# Definimos un pool de clientes extemporáneos que tienen una probabilidad positiva de comprar productos 
CLIENTES_EXTEMPORANEOS = simulaClientes(tasa_tota_nuevos_clientes)

# 5. Se simula el tiempo que se tarda el camión en llegar a su destino 
for i in range(tiempo): 
    j = 0 
    # Variable que va contando los clientes que en el tiempo t deciden comprar un producto
    cantidad_de_nuevos_clientes = 0 
    # Cantidad de descuentos asignados al tiempo i 
    descuentos_i = 0 

    for producto in productos["Nombre"]:
        # 5.1 Se simula la cantidad de nuevos clientes que van a comprar el producto en ese tiempo
        nuevos_clientes = simularDemanda(tasas_nuevos_clientes[producto])
        cantidad_de_nuevos_clientes += nuevos_clientes
        # Variable que regula la cantidad de nuevo producto solicitdado 
        nuevos_productos = 0 

        # 5.1 Por cada tiempo, se simula una cantidad de clientes que decidieron comprar el producto en ese tiempo  (se tiene que ver si hay productos disponibles)
        # 5.2 Por cada cliente se simula el método de descuento que se le va a aplicar
        # 5.3 Se actualizan los ingresos de ventas en total y por producto ya considerando los descuentos.

        clientes_compran = random.sample(CLIENTES_EXTEMPORANEOS, nuevos_clientes)
        # Extraemos la cantidad con la que se cuenta de ese producto 
        cantidad_del_producto = productos.loc[productos['Nombre'] == producto, 'cantidad'].values[0]
        # 2.1. Para cada cliente se simula el método de descuento que se le va a aplicar
        for cliente in clientes_compran:
            # Agregamos el producto al cliente 
            # Simulamos de una uniforme discreta de 0 a cantidad_promedio[producto]
            cantidad = np.random.randint(0, cantidad_promedio[producto])
            if(CANTIDADES_SOLICITADAS[producto] + cantidad <= cantidad_del_producto): 
                cliente.agregarProducto(producto, cantidad)
                # Aplicamos el descuento             
                cliente.agregarProducto(producto, cantidad)

                descuento, tipo_descuento = aplicaDescuentoIndividual(cliente, forma_a, forma_b, producto)
                # Vamos a guardar el descuento en un arreglo de descuentos para calcular posteriormente los ingresos por ventas (ya con el descuento aplicado)
                # Este arreglo nos facilitará el cálculo de los descuentos por producto para no iterar por todos los clientes
                DESCUENTOS_APLICADOS[producto].append(cantidad * (1 - descuento))
                # Agregamos las cantidades solicitadas 
                CANTIDADES_SOLICITADAS[producto] += cantidad
                nuevos_productos += cantidad
                # Vamos a registrar el método por descuento 
                if(tipo_descuento in METODOS_DESCUENTO):
                    METODOS_DESCUENTO[tipo_descuento] += 1
                elif(tipo_descuento != None):
                    METODOS_DESCUENTO[tipo_descuento] = 1
                
                # Agregamos el descuento 
                if(descuento != 0): 
                    descuentos_i += 1
            else: 
                # Si no hay suficiente producto, no se le vende al cliente 
                print(f'Se han agotado las existencias del producto {producto}')
                CANTIDADES_NO_SATISFECHAS[producto] += cantidad
        
        # Se actualiza el precio del producto y se disminuye en un factor porporcional a la variable de nuevos_productos (ACTUALIZAR)
        # Vamos a actualizarlo siempre y cuando el precio del producto sea mayor o igual al límite inferior
        if(productos.loc[productos["Nombre"] == producto, "precio"].tolist()[0] >= limite_inferior[producto]):
            # Actualización del precio 
            productos.loc[productos["Nombre"] == producto, "precio"] = ajusta_precio(productos.loc[productos["Nombre"] == producto, "precio"], nuevos_clientes)

        # Ajustamos los productos solicitados al dataframe productos 
        productos.loc[productos["Nombre"] == producto, "productos_solicitados"] = CANTIDADES_SOLICITADAS[producto]

        # Actualizamos las series de tiempo 
        cantidades_solicitadas_en_tiempo[producto].append(nuevos_productos)
        precios_por_producto_en_tiempo[producto].append(productos.loc[productos["Nombre"] == producto, "precio"].tolist()[0])
        
    # Calculamos los ingresos después de la actualización 
    ingresos, ingresos_x_producto = calculaIngresosConDescuento(DESCUENTOS_APLICADOS, productos)

    # Vamos a actualizar ingreos_por_producto_en_tiempo
    for producto in productos["Nombre"]:
        ingresos_por_producto_en_tiempo[producto].append(ingresos_x_producto[producto])

    ingresos_en_tiempo.append(ingresos)
    cantidad_de_descuentos_aplicados_tiempo.append(descuentos_i)
    clientes_nuevos_tiempo.append(cantidad_de_nuevos_clientes)


# Observaciones: 
# Cuando hayan más clientes que soliciten un producto, el precio va a ir disminuyendo 

display(productos)
# Resultados (por camión)

# Observaciones 
# El procedimiento anterior se hará por camión. Por lo que se pueden cambiar las variables globales que hacen referencia a las tasas de demanda, descuentos ofrecidos y capacidad de los camiones.
# Ya con estos datos, se puede aplicar una clusterización de usuarios para clasificarlos en grupos  que compran ciertos paquetes de productos. 

,Id,Nombre,precio,peso,dimensiones,cantidad,demanda_clientes,productos_solicitados,limite_inferior
0,0,Frijoles,9.047386,12,6,923,22,71,9.0
1,1,Leche,23.099367,12,9,824,77,115,22.5
2,2,Cereal,18.424111,17,8,949,6,88,18.0
3,3,Arroz,15.738985,18,1,858,44,156,15.3
4,4,Fruta,18.980721,13,6,841,11,186,18.9
5,5,Jugos,22.020210,16,9,784,52,233,21.6


# Gráficas de resultados

In [17]:
# 1. Gráfica del precio por producto a lo largo del tiempo 
fig = go.Figure()
for producto in cantidades_solicitadas_en_tiempo:
    fig.add_trace(go.Scatter(x=np.arange(len(cantidades_solicitadas_en_tiempo[producto][1:])), y=cantidades_solicitadas_en_tiempo[producto][1:], name=producto))

fig.update_layout(title="Cantidad de productos solicitados por producto a lo largo del tiempo", xaxis_title="Tiempo", yaxis_title="Cantidad de productos solicitados")
fig.update_layout(font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"))
fig.show()

# 2. Gráfica de los ingresos por ventas a lo largo del tiempo 
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ingresos_en_tiempo[1:])), y=ingresos_en_tiempo[1:], name="Ingresos"))
fig.update_layout(title="Ingresos por ventas a lo largo del tiempo", xaxis_title="Tiempo", yaxis_title="Ingresos")
fig.update_layout(font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"))
fig.show()

# 3. Gráfica de los ingresos por ventas por producto a lo largo del tiempo
fig = go.Figure()
for producto in ingresos_por_producto_en_tiempo:
    fig.add_trace(go.Scatter(x=np.arange(len(ingresos_por_producto_en_tiempo[producto][1:])), y=ingresos_por_producto_en_tiempo[producto][1:], name=producto))
fig.update_layout(title="Ingresos por ventas por producto a lo largo del tiempo", xaxis_title="Tiempo", yaxis_title="Ingresos")
fig.update_layout(font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"))
fig.show()

# 4. Gráfica de los precios por producto a lo largo del tiempo
fig = go.Figure()
for producto in precios_por_producto_en_tiempo:
    fig.add_trace(go.Scatter(x=np.arange(len(precios_por_producto_en_tiempo[producto])), y=precios_por_producto_en_tiempo[producto], name=producto))
    # Vamos a agregar una línea horizontal punteada que sea el límite inferior del precio del producto y que sea del mismo color que la línea del producto
    fig.add_shape(type="line", x0=0, y0=limite_inferior[producto], x1=len(precios_por_producto_en_tiempo[producto]), y1=limite_inferior[producto], line=dict(color=fig.data[-1].line.color, width=1, dash="dash"))
fig.update_layout(title="Precios por producto a lo largo del tiempo", xaxis_title="Tiempo", yaxis_title="Precios")
fig.update_layout(font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"))
fig.show()

# 5. Gráfica de la cantidad de clientes que en el tiempo decide comprar un producto
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(clientes_nuevos_tiempo)), y=clientes_nuevos_tiempo, name="Clientes"))
fig.update_layout(title="Cantidad de clientes que en el tiempo decide comprar un producto", xaxis_title="Tiempo", yaxis_title="Cantidad de clientes")
fig.update_layout(font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"))
fig.show()

# 6. Gráfica de pie para los métodos de descuento aplicados
fig = go.Figure(data=[go.Pie(labels=list(METODOS_DESCUENTO.keys()), values=list(METODOS_DESCUENTO.values()))])
fig.update_layout(title="Métodos de descuento aplicados")
fig.update_layout(font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"))
fig.show()

# 7. Gráfica de la cantidad de descuentos aplicados a lo largo del tiempo
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(cantidad_de_descuentos_aplicados_tiempo)), y=cantidad_de_descuentos_aplicados_tiempo, name="Descuentos"))
fig.update_layout(title="Cantidad de descuentos aplicados a lo largo del tiempo", xaxis_title="Tiempo", yaxis_title="Cantidad de descuentos")
fig.update_layout(font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"))
fig.show()

# 8. Gráfica de barras de los productos solicitados 
fig = go.Figure(data=[go.Bar(x=list(CANTIDADES_SOLICITADAS.keys()), y=list(CANTIDADES_SOLICITADAS.values()))], layout=go.Layout(title="Cantidad de productos solicitados"))
# Agregamos diferentes colores a la gráfica
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6)
fig.update_layout(font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"))
fig.show()

# 9. Gráfica de los productos que fueron solicitados y que no pudieron ser colocados debido a que el producto se había agoatado
fig = go.Figure(data=[go.Bar(x=list(CANTIDADES_NO_SATISFECHAS.keys()), y=list(CANTIDADES_NO_SATISFECHAS.values()))], layout=go.Layout(title="Cantidad de productos no satisfechos"))
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6)
fig.update_layout(font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"))
fig.show()

# Observaciones: 

Para las adquisiciones en el tiempo, primero se tienen que hacer puebas para llevar productos extra (ya con estudios estadísticos, se estima la demanda en el tiempo). Dicho proceso dependerá de 
la publicidad que se envíe de forma directa a clusters de personas cerca de las personas que ya adquirieron un producto. Este ejemplo no considera agregar nuevos productos considerando las restricciones. 

De forma alternativa, esta simulación se puede representar como un periodo de ventas antes de ingresar los productos al camión para su envío. Es decir, el el tiempo que se tarda el camión en llegar se convierte en el tiempo de espera antes de mandar un camión y se pueden ver como los clientes van adquiriendo productos en el tiempo y los precios se van asjutando. 